# Created by Diana Janik and Jan Markiewicz

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

In [2]:
x = np.load('classification_problem.npy')
y = []
class1 = np.array([1.,0.,0.])
class1 = np.tile(class1, (1000, 1))
y.extend(class1)
class2 = np.array([0.,1.,0.])
class2 = np.tile(class2, (1000, 1))
y.extend(class2)
class3 = np.array([0.,0.,1.])
class3 = np.tile(class3, (1000, 1))
y.extend(class3)
y = np.asarray(y)
y = y.astype(np.float32)
y = y[..., np.newaxis]

In [3]:
print(x.shape)
print(x)

(3000, 12)
[[ 983.99054  942.2865   661.72797 ...  372.1592   721.33167  573.65985]
 [ 977.8995   943.00995  660.28    ...  358.53558  717.0906   515.003  ]
 [ 997.3773   943.70807  663.5131  ...  349.9904   668.514    501.89447]
 ...
 [1053.4028   943.90314  654.8272  ...  353.91534  699.50146  514.19226]
 [1055.4299   942.15045  660.39075 ...  356.51056  706.0399   562.4285 ]
 [1023.4271   940.56964  663.16327 ...  349.66464  737.87274  586.15576]]


In [4]:
print(y.shape)
y = np.squeeze(y)
print(y.shape)

(3000, 3, 1)
(3000, 3)


In [6]:
values_with_labels = np.empty((3000,0), dtype=np.object)
values_with_labels = np.append(values_with_labels, np.array(x), axis=1)
values_with_labels = np.append(values_with_labels, np.array(y), axis=1)
print(values_with_labels)
pd.DataFrame(values_with_labels).to_csv('data.csv',index=False)

[[983.9905395507812 942.2864990234375 661.7279663085938 ... 1.0 0.0 0.0]
 [977.8994750976562 943.0099487304688 660.280029296875 ... 1.0 0.0 0.0]
 [997.3773193359375 943.7080688476562 663.5131225585938 ... 1.0 0.0 0.0]
 ...
 [1053.40283203125 943.9031372070312 654.8272094726562 ... 0.0 0.0 1.0]
 [1055.429931640625 942.1504516601562 660.3907470703125 ... 0.0 0.0 1.0]
 [1023.4271240234375 940.5696411132812 663.1632690429688 ... 0.0 0.0 1.0]]


In [8]:
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(12, 120)
        self.fc3 = nn.Linear(120, 3)
  
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return x

In [9]:
model = Model()

In [10]:
from torch import optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [11]:
for epoch in range(1, 11): ## run the model for 10 epochs
    train_loss, valid_loss = [], []

In [12]:
t_xs = torch.from_numpy(x)
t_ys = torch.from_numpy(y)
print(t_xs.shape)
print(t_ys.shape)
dataset = torch.utils.data.TensorDataset(t_xs, t_ys)
(train_set, valid_set) = torch.utils.data.random_split(dataset,(2000,1000))

torch.Size([3000, 12])
torch.Size([3000, 3])


In [13]:
print(train_set[0])

(tensor([993.2719, 943.9230, 647.6940, 928.0901, 870.9363, 105.6366,   2.5085,
        902.8651, 198.1741, 332.7198, 720.2141, 515.5229]), tensor([0., 1., 0.]))


In [15]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=int(len(train_set)/10))

In [16]:
optimizer = torch.optim.SGD(model.parameters(),lr =0.002)

In [17]:
loss_func = torch.nn.MSELoss()

In [18]:
from IPython import display
import time
err_train=[]
err_valid=[]
outputlines = []
for epoch in range(2000):    
    for datum in train_loader:
        optimizer.zero_grad()
        (features,target) = datum
        pred = model(features)
        loss = loss_func(pred, target)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        vpred  = model(valid_set[:][0])
        vloss  = loss_func(vpred,valid_set[:][1])
        err_valid.append(vloss)
        pred  = model(train_set[:][0])
        loss  = loss_func(pred,train_set[:][1])
        err_train.append(loss)
    if epoch%100 == 0:
        print("epoch %d %f %f " % (epoch, loss, vloss)) 
            

epoch 0 26037319680.000000 26037325824.000000 
epoch 100 1805944192.000000 1805945600.000000 
epoch 200 125260152.000000 125260536.000000 
epoch 300 8688038.000000 8688139.000000 
epoch 400 602601.437500 602628.000000 
epoch 500 41796.546875 41803.542969 
epoch 600 2899.212158 2901.055420 
epoch 700 201.295227 201.780930 
epoch 800 14.168521 14.296694 
epoch 900 1.189469 1.223485 
epoch 1000 0.289252 0.298470 
epoch 1100 0.226815 0.229503 
epoch 1200 0.222486 0.223454 
epoch 1300 0.222185 0.222701 
epoch 1400 0.222165 0.222561 
epoch 1500 0.222163 0.222528 
epoch 1600 0.222163 0.222519 
epoch 1700 0.222163 0.222517 
epoch 1800 0.222163 0.222517 
epoch 1900 0.222163 0.222516 
